In [1]:
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip HW4_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: HW4_data/public/train_6738.jpg  
 extracting: HW4_data/public/val_873.jpg  
 extracting: HW4_data/public/val_673.jpg  
 extracting: HW4_data/public/train_2361.jpg  
 extracting: HW4_data/public/train_1559.jpg  
 extracting: HW4_data/public/train_556.jpg  
 extracting: HW4_data/public/train_8835.jpg  
 extracting: HW4_data/public/train_8072.jpg  
 extracting: HW4_data/public/train_5806.jpg  
 extracting: HW4_data/public/train_336.jpg  
 extracting: HW4_data/public/train_4072.jpg  
 extracting: HW4_data/public/train_2778.jpg  
 extracting: HW4_data/public/train_1667.jpg  
 extracting: HW4_data/public/train_4722.jpg  
 extracting: HW4_data/public/train_13.jpg  
 extracting: HW4_data/public/train_6297.jpg  
 extracting: HW4_data/public/train_6576.jpg  
 extracting: HW4_data/public/train_2489.jpg  
 extracting: HW4_data/public/val_699.jpg  
 extracting: HW4_data/public/train_1589.jpg  
 extracting: HW4_data/public/train_4449.jp

In [4]:
X_train = []
for i in range(10000):
    X_train.append(np.asarray(Image.open("HW4_data/public/train_{i}.jpg".format(i=i))))

df_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)

In [5]:
X_val = []
for i in range(1000):
    X_val.append(np.asarray(Image.open("HW4_data/public/val_{i}.jpg".format(i=i))))
    
df_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)

In [6]:
X_test = []
for i in range(1500):
    X_test.append(np.asarray(Image.open("HW4_data/private/test_{i}.jpg".format(i=i))))

df_test_demo = pd.read_csv("HW4_data/private/test_demo.csv", index_col=0)

In [7]:
Y_train = np.array(df_train["Edema"])
Y_val = np.array(df_val["Edema"])

In [8]:
import tensorflow as tf
xTrain = tf.keras.utils.normalize(X_train, axis=1)
xVal = tf.keras.utils.normalize(X_val, axis=1)

In [9]:
print(xTrain.shape)
print(Y_train.shape)

print(xVal.shape)
print(Y_val.shape)

(10000, 128, 128)
(10000,)
(1000, 128, 128)
(1000,)


In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
import os
import cv2

In [11]:
model = Sequential()

model.add(Conv2D(16, 1, padding="same", input_shape=(128, 128, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Conv2D(32, 1, padding="same"))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Conv2D(64, 1, padding="same"))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      32        
                                                                 
 activation (Activation)     (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        544       
                                                                 
 activation_1 (Activation)   (None, 64, 64, 32)        0

In [13]:
history = model.fit(np.array(xTrain), np.array(Y_train), batch_size=64, verbose=1, epochs = 10, validation_data = (np.array(xVal), np.array(Y_val)), shuffle=False)

Epoch 1/10
157/157 [==============================] - 65s 410ms/step - loss: 0.8193 - accuracy: 0.7245 - val_loss: 1.1436 - val_accuracy: 0.2820
Epoch 2/10
157/157 [==============================] - 65s 411ms/step - loss: 0.4849 - accuracy: 0.7730 - val_loss: 0.7862 - val_accuracy: 0.2820
Epoch 3/10
157/157 [==============================] - 64s 411ms/step - loss: 0.4435 - accuracy: 0.7928 - val_loss: 0.8347 - val_accuracy: 0.2930
Epoch 4/10
157/157 [==============================] - 64s 409ms/step - loss: 0.4288 - accuracy: 0.7987 - val_loss: 0.4782 - val_accuracy: 0.8010
Epoch 5/10
157/157 [==============================] - 65s 411ms/step - loss: 0.4319 - accuracy: 0.7962 - val_loss: 0.4499 - val_accuracy: 0.8070
Epoch 6/10
157/157 [==============================] - 64s 410ms/step - loss: 0.4160 - accuracy: 0.8039 - val_loss: 0.4247 - val_accuracy: 0.8140
Epoch 7/10
157/157 [==============================] - 64s 410ms/step - loss: 0.4080 - accuracy: 0.8036 - val_loss: 0.4859 - val_ac

In [14]:
tmpY_test = []
Y_t = []
Y_test = []
for i in range(len(X_test)):
  img = X_test[i]
  input_img = np.expand_dims(img, axis=0)
  tmpY_test.append(model.predict(input_img))

for i in range(len(tmpY_test)):
  Y_t.append([i, int(tmpY_test[i][0])])
for i in range(len(Y_t)):
  Y_test.append(Y_t[i])
print(Y_test)

[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [13, 0], [14, 0], [15, 0], [16, 0], [17, 0], [18, 0], [19, 0], [20, 1], [21, 0], [22, 0], [23, 0], [24, 0], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [30, 0], [31, 0], [32, 0], [33, 1], [34, 0], [35, 0], [36, 0], [37, 0], [38, 1], [39, 0], [40, 0], [41, 0], [42, 0], [43, 0], [44, 0], [45, 0], [46, 0], [47, 0], [48, 0], [49, 0], [50, 0], [51, 0], [52, 0], [53, 0], [54, 0], [55, 1], [56, 0], [57, 0], [58, 1], [59, 0], [60, 1], [61, 0], [62, 0], [63, 0], [64, 0], [65, 0], [66, 0], [67, 0], [68, 0], [69, 1], [70, 0], [71, 0], [72, 0], [73, 0], [74, 0], [75, 0], [76, 1], [77, 0], [78, 0], [79, 0], [80, 0], [81, 0], [82, 0], [83, 1], [84, 0], [85, 0], [86, 1], [87, 0], [88, 0], [89, 0], [90, 0], [91, 0], [92, 0], [93, 0], [94, 0], [95, 0], [96, 0], [97, 0], [98, 0], [99, 0], [100, 0], [101, 0], [102, 0], [103, 0], [104, 0], [105, 1], [106, 0], [107, 0], [108, 0], [109, 0], [110, 0],

In [15]:
tmpY_p = []
Y_p = []
for i in range(len(X_val)):
  img = X_val[i]
  input_img = np.expand_dims(img, axis=0)
  tmpY_p.append(model.predict(input_img))

for i in range(len(tmpY_p)):
  Y_p.append(int(tmpY_p[i][0]))
print(Y_p)

[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [16]:
import csv
studentID = '108062281'
output_dataroot = studentID + '_basic_prediction.csv'
output_datalist = np.array(Y_test)
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['', 'Edema'])

  for row in output_datalist:
    writer.writerow(row)